In [ ]:
# create the word2vec dict from the dictionary

import random as rand
import pandas as pd
from numpy import array
from numpy import cumsum
import scipy
import tensorflow
from keras import Sequential
from keras.layers import *
from keras.models import *
import gensim

embedding_path = "wikipedia-pubmed-and-PMC-w2v.bin"
#embedding_path = "glove.6B.100d.txt"
#embedding_path = "glove.twitter.27B.100d.txt" ## change 

def get_word2vec(file_path):
    file = open(embedding_path, "r", encoding="utf-8")
    if (file):
        word2vec = dict()
        split = file.read().splitlines()
        for line in split:
            key = line.split(' ',1)[0] # the first word is the key
            value = np.array([float(val) for val in line.split(' ')[1:]])
            word2vec[key] = value
        return (word2vec)
    else:
        print("invalid file path")
        
def load_bin_vec(fname): 
    """ 
    Loads 300x1 word vecs from Google (Mikolov) word2vec 
    """ 
    word_vecs = {} 
    with open(fname, "rb") as f: 
        header = f.readline() 
        vocab_size, layer1_size = map(int, header.split()) 
        binary_len = np.dtype('float32').itemsize * layer1_size 
        for line in range(vocab_size): 
            word = [] 
            while True: 
                ch = f.read(1) 
                if ch == ' ': 
                    word = ''.join(word) 
                    break 
                if ch != '\n': 
                    word.append(ch)   
            word_vecs[word] = np.fromstring(f.read(binary_len), dtype='float32')   
    return word_vecs

#w2v = get_word2vec(embedding_path)
#w2v = load_bin_vec(embedding_path)

MemoryError: 

In [ ]:
import gensim
#model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)


ModuleNotFoundError: No module named 'gensim'

In [ ]:
# Test harness for word2vec function

import random

for i in range(10):
    sample = random.choice(list(w2v.keys()))
    print(len(w2v[sample]))
    print(w2v[sample])
    print(sample)
    
print(len(w2v.keys()))

NameError: name 'w2v' is not defined

In [ ]:
# Loading in our indication data; returns a dataframe and a dictionary[sentence] = label

def load_FDA_data():
    file = open("paranoia4_no_commas.csv", 'r')
    data_dict = {}
    df = pd.DataFrame()
    for line in file:
        current_line = line.split(',')[-2:]
        if len(current_line) == 2:
            key = current_line[0]
            item = current_line[1].strip()
            data_dict[key] = item
            temp_series = pd.Series([current_line[0], current_line[1].strip()])
            df = df.append(temp_series, ignore_index=True)
    return df, data_dict

df, data_dict = load_FDA_data()
print(df)



                                                      0  1
0      minocycline hydrochloride tablets are indicat...  1
1     respiratory tract infections caused by mycopla...  1
2     lymphogranuloma venereum caused by chlamydia t...  1
3     psittacosis ( ornithosis ) due to chlamydia ps...  1
4     trachoma caused by chlamydia trachomatis  alth...  1
5     inclusion conjunctivitis caused by chlamydia t...  1
6     nongonococcal urethritis  endocervical  or rec...  1
7           relapsing fever due to borrelia recurrentis  1
8     chancroid caused by haemophilus ducreyi plague...  1
9               tularemia due to francisella tularensis  1
10                    cholera caused by vibrio cholerae  1
11    campylobacter fetus infections caused by campy...  1
12    brucellosis due to brucella species ( in conju...  1
13        bartonellosis due to bartonella bacilliformis  1
14    granuloma inguinale caused by calymmatobacteri...  1
15    minocycline is indicated for treatment of infe... 

In [ ]:
# Tokenizing (breaking sentences into words. 
# Gets tricky because there are a few different Tokenizer vs Tokenize classes from different packages.

import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
# nltk.download('wordnet')
# nltk.download('stopwords')
from nltk.tokenize import *
from nltk.corpus import wordnet as wn
tknzr = TweetTokenizer()


def get_tokens(sentence):
#     tokens = nltk.word_tokenize(sentence)  # now using tweet tokenizer
    tokens = tknzr.tokenize(sentence)
    tokens = [token for token in tokens if (token not in stopwords and len(token) > 1)]
    tokens = [get_lemma(token) for token in tokens]
    return (tokens)


def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
    
token_list = (df[0].apply(get_tokens))

In [ ]:
# Quick test harness for tokenization functions.

#print(type(token_list))
#print("")
#print(token_list)
#print("")
#print(tknzr)

l = token_list.tolist()

flat_list = []
for sublist in l:
    for item in sublist:
        flat_list.append(item)
        
pd.Series(flat_list).value_counts()


indicate         3339
treatment        3001
patient          2852
use              1917
therapy           980
adult             972
infection         843
disease           765
tablet            734
see               698
drug              676
active            655
clinical          611
study             585
combination       581
include           530
cause             524
symptom           519
acute             512
injection         492
associate         480
ingredient        471
years             467
may               467
age               465
risk              464
chronic           458
unii              457
usp               442
limitation        435
                 ... 
nebulization        1
generator           1
dizzy               1
anuric              1
expander            1
ahealon             1
6940                1
student             1
bloom               1
carmine             1
infract             1
living              1
protector           1
safely              1
viper     

In [ ]:
# Converting tokens to integers. Beware Tokenizer from keras (vs tokenize from nltk).
# If there are bug(s), I'd expect one to be here. This might all be redundant too, as another Tokenizer is used in the model.

from keras.preprocessing.text import Tokenizer
import keras
tokenizer = Tokenizer(num_words = 850)  # change num_words as needed
tokenizer.fit_on_texts(df[0].tolist())  # no idea what this line is doing. maybe building a vocabulary?
sequences = tokenizer.texts_to_sequences(df[0].tolist())

encoded_docs = sequences  
# pad documents to a max length of 4 words
X = keras.preprocessing.sequence.pad_sequences(sequences, padding='post')

print(X.shape)
_, max_len = X.shape
print(max_len)

(7864, 189)
189


In [ ]:
# Test Harness for token--> integer conversion


#from random import *

#print(len(sequences))

#for i in range(10):
 #   print(sequences[rand.randint(1, len(sequences)-1)])
#print(len(X))
#for i in range(100):
 #   print(len(X[rand.randint(1, len(X)-1)]))

In [ ]:
# Splits into training and testing data. Also reduces the dataset to a binary. Worth noting that all sentences marked "unsure"
# were here marked as not indications, so only clear indications receive a 1 and all else is 0.

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import *
from sklearn.svm import *
from sklearn.metrics import *

#le = preprocessing.LabelEncoder()
#le.classes_ = 3
#Y_new = le.fit_transform(df[1])
Y_new = array(df[1].replace({'u': '0', 'c': '0', '2': '0', '3': '0', '4': '0', '':'0', '"Active Ingredient UNII(s)':'0', '/':'0'}))  # should not need to do this once you have polished data
Y_new = Y_new.astype(int)

X_train, X_test, Y_train, Y_test =  train_test_split(X, Y_new, test_size=0.20, shuffle=True)

In [ ]:
# Testing if the shuffling is actually working.
# Spoiler: it is.

from sklearn.model_selection import train_test_split

a1 = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
a2 = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

X_tr_fake, X_te_fake, Y_tr_fake, Y_te_fake = train_test_split(a1, a2, test_size=0.2, shuffle=True)
print(X_tr_fake, X_te_fake, "\n", Y_tr_fake, Y_te_fake)


[ 4  1  7  8 10  2  5  3] [9 6] 
 [ 4  1  7  8 10  2  5  3] [9 6]


In [ ]:
# Another test harness to make sure the above is actually splitting data, and to make sure shapes of arrays feed into
# the model properly.

print(X_train.shape)
print(X_test.shape)
print(df[1].max())
print(X_train)
print("Y_new:")
print(Y_new)
Y_new.min()
pd.Series(Y_new).value_counts()

(6291, 189)
(1573, 189)
u
[[166  74 292 ...   0   0   0]
 [308  15  19 ...   0   0   0]
 [537   3 658 ...   0   0   0]
 ...
 [534 320   4 ...   0   0   0]
 [  5  78 192 ...   0   0   0]
 [ 32  48  50 ...   0   0   0]]
Y_new:
[1 1 1 ... 0 0 0]


1    4109
0    3755
dtype: int64

In [ ]:
# This runs an SVM on the data. Takes a while to run.


def svm_wrapper(X_train,Y_train):
    param_grid = [
    {'C': [1, 10], 'kernel': ['linear']},
    {'C': [1, 10], 'gamma': [0.1,0.01], 'kernel': ['rbf']},]
    svm = GridSearchCV(SVC(),param_grid)
    svm.fit(X_train, Y_train)
    return(svm)

#svm = svm_wrapper(X_train,Y_train)
#Y_pred = svm.predict(X_test)
#score = accuracy_score(Y_test,Y_pred)
#print("accuracy :", score)

In [ ]:
# This is the main function itself. 

from sklearn.metrics import classification_report,confusion_matrix
from numpy import zeros

t = Tokenizer()
t.fit_on_texts(token_list)
vocab_size = len(t.word_index) + 1

### Below block is commented out as I think this was already done.

# integer encode the documents
# encoded_docs = t.texts_to_sequences(token_list)
# pad documents to a max length of 4 words
# max_length = max_len = 10
# X = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# y = Y_new

embedding_matrix = zeros((vocab_size, 100))  # 100 because the original data is a 100 vector

#  This is creating an embedding matrix, but not quite sure what or why.
for word, i in t.word_index.items():
    embedding_vector = w2v.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

        
### Main Model Building
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


input = Input(shape=(max_len,))
model = Embedding(vocab_size,100,weights=[embedding_matrix],input_length=max_len)(input)
model = Bidirectional(LSTM(100,return_sequences=True,dropout=0.50),merge_mode='concat')(model)
model = TimeDistributed(Dense(100,activation='relu'))(model)
model = Flatten()(model)
model = Dense(100,activation='relu', kernel_regularizer=regularizers.l2(.01))(model)
output = Dense(1,activation='sigmoid')(model)
model = Model(input,output)
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
# give a loss as the model is working
model.fit(X_train, Y_train, verbose=1, epochs=3)
#print('Accuracy: %f' % (accuracy*100))

Y_pred = model.predict(X_test, verbose=1) # scrutinize!
y_pred = np.array([np.argmax(pred) for pred in Y_pred])

print('  Classification Report:\n',classification_report(Y_test, Y_pred.round()),'\n')
print('Confusion matrix:', confusion_matrix(Y_test, Y_pred.round()))



model.summary()

#print(" You can safely ignore the \'2' column zeroes as they just say there was no data for that class. I ran a category classifier on a binary problem.")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
6291/6291 [==============================] - 88s 14ms/step - loss: 0.5827 - acc: 0.7697
Epoch 2/3
6291/6291 [==============================] - 86s 14ms/step - loss: 0.3196 - acc: 0.8822
Epoch 3/3
1573/1573 [==============================] - 5s 3ms/step
  Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.91      0.91       743
           1       0.92      0.92      0.92       830

   micro avg       0.92      0.92      0.92      1573
   macro avg       0.92      0.92      0.92      1573
weighted avg       0.92      0.92      0.92      1573
 

Confusion matrix: [[677  66]
 [ 67 763]]
_________________________________________________________________
Layer (type)                 Output Shap

In [ ]:
print(X.shape)
print(Y_train.shape)
print(Y_test.shape)
pd.Series(Y_test).value_counts()

(7864, 189)
(6291,)
(1573,)


1    830
0    743
dtype: int64

In [ ]:
#print('  Classification Report:\n',classification_report(Y_test, y_pred),'\n')
#print('Confusion matrix:', confusion_matrix(Y_test, y_pred))

print('  Classification Report:\n',classification_report(Y_test, Y_pred.round()),'\n')
print('Confusion matrix:', confusion_matrix(Y_test, Y_pred.round()))

  Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.91      0.91       743
           1       0.92      0.92      0.92       830

   micro avg       0.92      0.92      0.92      1573
   macro avg       0.92      0.92      0.92      1573
weighted avg       0.92      0.92      0.92      1573
 

Confusion matrix: [[677  66]
 [ 67 763]]
